In [1]:
class Route:
    def __init__(self, route_id, duration=1.5):
        """
        Конструктор для маршрута автобуса.

        :param route_id: Идентификатор маршрута.
        :param duration: Длительность маршрута в часах. По умолчанию 1.5 часа.
        """
        self.route_id = route_id
        self.duration = duration  # Время в пути по маршруту (в часах)


In [2]:
class Driver:
    def __init__(self, driver_id):
        """
        Конструктор для водителя.

        :param driver_id: Идентификатор водителя.
        """
        self.driver_id = driver_id
        self.shifts = {day: [] for day in range(7)}  # Список смен по дням недели (0 - Понедельник, 6 - Воскресенье)
        self.work_hours = {day: 0 for day in range(7)}  # Рабочие часы по дням недели
        self.breaks = {day: [] for day in range(7)}  # Перерывы по дням недели

    def assign_shift(self, day, shift):
        """
        Назначить водителю смену для конкретного дня недели.

        :param day: День недели (0 - Понедельник, 6 - Воскресенье).
        :param shift: Смена для назначения.
        """
        self.shifts[day].append(shift)
        self.work_hours[day] += shift.work_hours
        self.breaks[day].extend(shift.breaks)

    def get_total_work_hours(self, day=None):
        """
        Возвращает общее количество рабочих часов водителя.
        Если день указан, возвращает для конкретного дня.
        """
        if day is not None:
            return self.work_hours[day]
        return sum(self.work_hours.values())

    def get_total_breaks(self, day=None):
        """
        Возвращает общее время перерывов водителя.
        Если день указан, возвращает для конкретного дня.
        """
        if day is not None:
            return sum(self.breaks[day])
        return sum([sum(b) for b in self.breaks.values()])

    def generate_driver_schedule(self):
        """
        Генерирует расписание для водителя на 7 дней недели с учётом смен.
        Смена для разных дней недели может быть разной длительности.
        """
        time_blocks = [
            ('00:00', '08:00', 8, [60]),   # Смена 1: с 00:00 до 08:00 (перерыв 1 час)
            ('08:00', '16:00', 8, [60]),   # Смена 2: с 08:00 до 16:00 (перерыв 1 час)
            ('16:00', '00:00', 8, [60]),   # Смена 3: с 16:00 до 00:00 (перерыв 1 час)
            ('00:00', '12:00', 12, [10, 10]),   # Смена 4: с 00:00 до 12:00 (перерывы 10 минут каждые 2-4 часа)
            ('12:00', '00:00', 12, [10, 10]),   # Смена 5: с 12:00 до 00:00 (перерывы 10 минут каждые 2-4 часа)
        ]

        # Размещение смен для каждого дня недели
        for day in range(7):  # Для каждого дня недели
            # Выбираем случайным образом смены для каждого дня
            shift = random.choice(time_blocks)
            start_time, end_time, work_hours, breaks = shift
            shift_obj = Shift(start_time=start_time, end_time=end_time, work_hours=work_hours, breaks=breaks)
            self.assign_shift(day, shift_obj)

    def print_schedule(self):
        """
        Печать расписания водителя по дням недели с временем начала и конца смены.
        """
        days_of_week = ["Понедельник", "Вторник", "Среда", "Четверг", "Пятница", "Суббота", "Воскресенье"]

        print(f"Driver {self.driver_id}:")
        for day in range(7):
            print(f"{days_of_week[day]} - Количество часов работы: {self.get_total_work_hours(day)}, Перерыв(ы) (минуты): {self.get_total_breaks(day)}")
            for shift in self.shifts[day]:
                print(f"  Смена: {shift.start_time} до {shift.end_time} | Рабочие часы: {shift.work_hours} | Перерыв(ы): {shift.breaks}")


class Shift:
    def __init__(self, start_time, end_time, work_hours, breaks=[]):
        """
        Конструктор для смены водителя.

        :param start_time: Время начала смены.
        :param end_time: Время окончания смены.
        :param work_hours: Продолжительность работы в часах.
        :param breaks: Список перерывов в минутах.
        """
        self.start_time = start_time
        self.end_time = end_time
        self.work_hours = work_hours  # Рабочие часы (например, 8 или 12 часов)
        self.breaks = breaks  # Перерывы (например, [60] для 60 минут)

    def calculate_total_break_time(self):
        """
        Возвращает общее время перерывов для данной смены.
        """
        return sum(self.breaks)

    def is_valid(self):
        """
        Проверка корректности смены:
        - Рабочие часы не превышают максимального времени (8 или 12 часов).
        - Перерывы не превышают допустимых норм.
        """
        if self.work_hours not in [8, 12]:
            return False  # Рабочая смена должна быть 8 или 12 часов
        if self.work_hours == 8 and sum(self.breaks) != 60:
            return False  # Для 8-часовой смены перерыв должен составлять 1 час
        if self.work_hours == 12 and not (10 <= sum(self.breaks) <= 20):
            return False  # Для 12-часовой смены перерывы должны быть 10-20 минут
        return True


In [3]:
import random

# Создание водителей и генерация расписания
drivers = [Driver(i) for i in range(1, 20)]  # 10 водителей
for driver in drivers:
    driver.generate_driver_schedule()
    driver.print_schedule()


Driver 1:
Понедельник - Количество часов работы: 8, Перерыв(ы) (минуты): 60
  Смена: 16:00 до 00:00 | Рабочие часы: 8 | Перерыв(ы): [60]
Вторник - Количество часов работы: 8, Перерыв(ы) (минуты): 60
  Смена: 16:00 до 00:00 | Рабочие часы: 8 | Перерыв(ы): [60]
Среда - Количество часов работы: 8, Перерыв(ы) (минуты): 60
  Смена: 08:00 до 16:00 | Рабочие часы: 8 | Перерыв(ы): [60]
Четверг - Количество часов работы: 12, Перерыв(ы) (минуты): 20
  Смена: 12:00 до 00:00 | Рабочие часы: 12 | Перерыв(ы): [10, 10]
Пятница - Количество часов работы: 8, Перерыв(ы) (минуты): 60
  Смена: 08:00 до 16:00 | Рабочие часы: 8 | Перерыв(ы): [60]
Суббота - Количество часов работы: 8, Перерыв(ы) (минуты): 60
  Смена: 16:00 до 00:00 | Рабочие часы: 8 | Перерыв(ы): [60]
Воскресенье - Количество часов работы: 8, Перерыв(ы) (минуты): 60
  Смена: 16:00 до 00:00 | Рабочие часы: 8 | Перерыв(ы): [60]
Driver 2:
Понедельник - Количество часов работы: 8, Перерыв(ы) (минуты): 60
  Смена: 08:00 до 16:00 | Рабочие часы: 8

In [4]:
from datetime import datetime, timedelta

class Bus:
    def __init__(self, bus_id, route, schedule=None):
        """
        Конструктор для автобуса.

        :param bus_id: Идентификатор автобуса.
        :param route: Объект маршрута.
        :param schedule: Список расписания для данного автобуса (если задано).
        """
        self.bus_id = bus_id
        self.route = route  # Связанный маршрут
        self.schedule = schedule if schedule else []  # Расписание автобуса

    def calculate_arrival_time(self, start_time_str):
        """
        Рассчитывает время прибытия автобуса на конечную остановку с учетом времени маршрута.

        :param start_time_str: Время отправления автобуса (в формате HH:MM).
        :return: Время прибытия на конечную остановку.
        """
        start_time = datetime.strptime(start_time_str, "%H:%M")
        travel_duration = timedelta(hours=1.5)  # Фиксированная длительность маршрута 1.5 часа
        arrival_time = start_time + travel_duration
        return arrival_time.strftime("%H:%M")

    def generate_schedule_for_shift(self, start_time_str, end_time_str):
        """
        Генерирует расписание для автобуса на основе времени смены с учётом циклов.

        :param start_time_str: Время начала смены (в формате HH:MM).
        :param end_time_str: Время окончания смены (в формате HH:MM).
        """
        schedule = []
        start_time = datetime.strptime(start_time_str, "%H:%M")
        end_time = datetime.strptime(end_time_str, "%H:%M")

        # Циклически генерируем отправления и прибытия, но не пропускаем окончание смены
        while start_time + timedelta(hours=1.5) < end_time:
            departure_time = start_time.strftime("%H:%M")
            arrival_time = self.calculate_arrival_time(departure_time)
            schedule.append((departure_time, arrival_time))
            start_time += timedelta(hours=1.5)  # Увеличиваем время на 1.5 часа

        return schedule

    def print_schedule(self):
        """
        Напечатает расписание автобуса с временем отправления и прибытия.
        """
        print(f"Bus {self.bus_id} Schedule:")
        for shift in self.schedule:
            print(f"  Смена: {shift['start_time']} до {shift['end_time']} | Часы работы: {shift['work_hours']} | Перерыв(ы): {shift['breaks']}")
            # Печатаем отправления и прибытия
            for dep, arr in shift['schedule']:
                print(f"    Отправление с конечной: {dep} | Прибытие на конечную: {arr}")

    def generate_schedule_for_24h(self):
        """
        Генерирует расписание автобуса на 24 часа.
        Учитывает пики и перерывы.
        """
        time_blocks = [
            ('00:00', '08:00', 8, [60]),   # Смена 1: с 00:00 до 08:00 (перерыв 1 час)
            ('08:00', '16:00', 8, [60]),   # Смена 2: с 08:00 до 16:00 (перерыв 1 час)
            ('16:00', '00:00', 8, [60]),   # Смена 3: с 16:00 до 00:00 (перерыв 1 час)
        ]

        for block in time_blocks:
            start_time, end_time, work_hours, breaks = block
            schedule_for_shift = self.generate_schedule_for_shift(start_time, end_time)
            self.schedule.append({
                'start_time': start_time,
                'end_time': end_time,
                'work_hours': work_hours,
                'breaks': breaks,
                'schedule': schedule_for_shift
            })


In [5]:
# Создание маршрута с длительностью 1.5 часа
route = Route(route_id=1, duration=1.5)

# Создание автобусов и добавление смен
buses = [Bus(bus_id=i, route=route) for i in range(1, 9)]  # 8 автобусов
for bus in buses:
    bus.generate_schedule_for_24h()
    bus.print_schedule()


Bus 1 Schedule:
  Смена: 00:00 до 08:00 | Часы работы: 8 | Перерыв(ы): [60]
    Отправление с конечной: 00:00 | Прибытие на конечную: 01:30
    Отправление с конечной: 01:30 | Прибытие на конечную: 03:00
    Отправление с конечной: 03:00 | Прибытие на конечную: 04:30
    Отправление с конечной: 04:30 | Прибытие на конечную: 06:00
    Отправление с конечной: 06:00 | Прибытие на конечную: 07:30
  Смена: 08:00 до 16:00 | Часы работы: 8 | Перерыв(ы): [60]
    Отправление с конечной: 08:00 | Прибытие на конечную: 09:30
    Отправление с конечной: 09:30 | Прибытие на конечную: 11:00
    Отправление с конечной: 11:00 | Прибытие на конечную: 12:30
    Отправление с конечной: 12:30 | Прибытие на конечную: 14:00
    Отправление с конечной: 14:00 | Прибытие на конечную: 15:30
  Смена: 16:00 до 00:00 | Часы работы: 8 | Перерыв(ы): [60]
Bus 2 Schedule:
  Смена: 00:00 до 08:00 | Часы работы: 8 | Перерыв(ы): [60]
    Отправление с конечной: 00:00 | Прибытие на конечную: 01:30
    Отправление с конечн

In [6]:
import random
from collections import defaultdict

class Driver:
    def __init__(self, driver_id):
        self.driver_id = driver_id
        self.assigned_shifts = []  # Список смен, которые назначены водителю

    def assign_shift(self, time_slot):
        """Назначает водителю смену."""
        self.assigned_shifts.append(time_slot)

    def remove_shift(self, time_slot):
        """Удаляет назначенную смену у водителя."""
        self.assigned_shifts.remove(time_slot)


class Chromosome:
    def __init__(self, time_slots, drivers):
        """
        time_slots: Список временных слотов (дни недели + часы).
        drivers: Список водителей.
        """
        self.genes = []  # Список кортежей (time_slot, driver_id)
        for time_slot in time_slots:
            driver = random.choice(drivers)
            # Водитель может работать смены длиной 8 или 12 часов
            shift_duration = random.choice([8, 12])  # Выбираем длительность смены
            if self._is_valid_shift(time_slot, shift_duration):
                self.genes.append((time_slot, driver.driver_id, shift_duration))
                driver.assign_shift(time_slot)

    def _is_valid_shift(self, start_time, shift_duration):
        """Проверяет, возможно ли назначить смену длиной shift_duration начиная с start_time."""
        day, hour = start_time
        end_time = hour + shift_duration
        if end_time > 24:
            return False  # Смена не может выходить за пределы 24 часов
        return True

    def fitness(self):
        """
        Целевая функция: минимизировать количество занятых водителей.
        """
        # Уникальные водители в расписании
        unique_drivers = {gene[1] for gene in self.genes}
        return len(unique_drivers)

    def crossover(self, other):
        crossover_point = random.randint(1, len(self.genes) - 1)
        child1 = Chromosome(time_slots, drivers)
        child2 = Chromosome(time_slots, drivers)
        child1.genes[:crossover_point] = self.genes[:crossover_point]
        child1.genes[crossover_point:] = other.genes[crossover_point:]
        child2.genes[:crossover_point] = other.genes[:crossover_point]
        child2.genes[crossover_point:] = self.genes[crossover_point:]
        return child1, child2

    def mutate(self, mutation_rate):
        for i in range(len(self.genes)):
            if random.random() < mutation_rate:
                new_driver = random.choice(drivers)
                time_slot, shift_duration = self.genes[i][0], self.genes[i][2]
                old_driver = next(filter(lambda d: d.driver_id == self.genes[i][1], drivers))
                old_driver.remove_shift(time_slot)
                new_driver.assign_shift(time_slot)
                self.genes[i] = (time_slot, new_driver.driver_id, shift_duration)


def genetic_algorithm(population_size, generations, mutation_rate, time_slots, drivers):
    """
    Реализует генетический алгоритм для минимизации количества водителей.
    """
    population = [Chromosome(time_slots, drivers) for _ in range(population_size)]

    for generation in range(generations):
        fitness_scores = [chromosome.fitness() for chromosome in population]

        # Отбор
        new_population = []
        for _ in range(population_size):
            parents = random.sample(population, 2)
            parent1 = min(parents, key=lambda x: x.fitness())
            new_population.append(parent1)

        # Кроссинговер и мутация
        for i in range(0, population_size, 2):
            if i + 1 < population_size:
                child1, child2 = new_population[i].crossover(new_population[i+1])
                child1.mutate(mutation_rate)
                child2.mutate(mutation_rate)
                new_population[i] = child1
                new_population[i + 1] = child2

        population = new_population

    best_chromosome = min(population, key=lambda x: x.fitness())
    return best_chromosome


# Определяем временные слоты (например, по часам)
week_days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
time_slots = [(day, hour) for day in week_days for hour in range(24)]  # 7 дней * 24 часа = 168 временных слотов

# Моделируем часы пик
peak_hours = {("Monday", 8), ("Monday", 18), ("Tuesday", 8), ("Tuesday", 18), ("Wednesday", 8), ("Wednesday", 18),
              ("Thursday", 8), ("Thursday", 18), ("Friday", 8), ("Friday", 18)}  # Пример: часы пик с 8 до 9 и 18 до 19

# Создаем водителей
drivers = [Driver(i) for i in range(1, 20)]  # Максимум 20 водителей

# Запуск генетического алгоритма
best_schedule = genetic_algorithm(population_size=100, generations=100, mutation_rate=0.1, time_slots=time_slots, drivers=drivers)

driver_schedules = defaultdict(list)
for time_slot, driver_id, shift_duration in best_schedule.genes:
    driver_schedules[driver_id].append((time_slot, shift_duration))

count_drivers = max(driver_schedules.keys())
print(f'Число водителей: {count_drivers}')


# Анализ результата
print("Лучший график:")


for driver_id, shifts in driver_schedules.items():
    print(f"Водитель {driver_id}: {shifts}")


Число водителей: 19
Лучший график:
Водитель 17: [(('Monday', 0), 8), (('Monday', 2), 8), (('Monday', 6), 12), (('Wednesday', 16), 8), (('Saturday', 0), 8), (('Sunday', 6), 12)]
Водитель 11: [(('Monday', 1), 12), (('Tuesday', 7), 12), (('Thursday', 6), 12), (('Friday', 2), 12), (('Sunday', 9), 12)]
Водитель 18: [(('Monday', 3), 12), (('Monday', 8), 12), (('Monday', 9), 8), (('Monday', 12), 8), (('Tuesday', 8), 12), (('Tuesday', 11), 8), (('Wednesday', 2), 8), (('Friday', 3), 12), (('Friday', 3), 8), (('Sunday', 0), 12), (('Sunday', 15), 8)]
Водитель 14: [(('Monday', 4), 12), (('Tuesday', 9), 8), (('Wednesday', 7), 8), (('Sunday', 4), 8)]
Водитель 1: [(('Monday', 5), 8), (('Tuesday', 1), 12), (('Wednesday', 0), 12), (('Sunday', 0), 8), (('Sunday', 14), 8)]
Водитель 13: [(('Monday', 7), 8), (('Tuesday', 5), 12), (('Wednesday', 1), 12), (('Saturday', 2), 8), (('Saturday', 11), 12)]
Водитель 15: [(('Monday', 10), 12), (('Tuesday', 6), 8), (('Thursday', 1), 12), (('Wednesday', 12), 8), (('Fr